In [32]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdn
import pyarrow
import os
import sys
import json
import urllib.request
from google.cloud import bigquery
from google.cloud.bigquery import job
import copy
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
dataset_id = 'airbridge_mart'
table_id = 'web_keyfeatures'
table_ref = f'ballosodeuk.{dataset_id}.{table_id}'

In [9]:
client = bigquery.Client()
query = f"""
SELECT 
  Event_Date, Cookie_ID, Platform ,Event_Category,
  i.Client_IP_subdivision, i.Client_IP_City,
  d.Label, d.Action, d.Value, d.productListID, d.query, d.IsRenewal,
  prd.name, prd.price, prd.position
FROM `ballosodeuk.airbridge_mart.web_keyfeatures`, UNNEST(e) as e, UNNEST(e.d) as d, UNNEST(products_str) as prd
WHERE Event_Date Between "2024-06-01" AND "2024-07-31"
"""

job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

df = query_job.to_dataframe()

In [31]:
product_col = 'name'

In [42]:
pd.read_csv("../../cp_data.csv").iloc[3:6]

,media,id,secret
3,nv_api1,DhLceR4k10A9Mqi3kT4U,61w6F8i7_L
4,nv_api2,dh3tneEl0VrBHBb57mF4,Wc_Fhp_IYJ
5,nv_api3,dn3KJfpROnI81EiM7Hou,BSMufjjtrN


In [43]:
id_1 = pd.read_csv("../../cp_data.csv").iloc[3:6].id[3]
sc_1 = pd.read_csv("../../cp_data.csv").iloc[3:6].secret[3]
id_2 = pd.read_csv("../../cp_data.csv").iloc[3:6].id[4]
sc_2 = pd.read_csv("../../cp_data.csv").iloc[3:6].secret[4]
id_3 = pd.read_csv("../../cp_data.csv").iloc[3:6].id[5]
sc_3 = pd.read_csv("../../cp_data.csv").iloc[3:6].secret[5]

In [46]:
import pandas as pd
import urllib.request
import urllib.parse
import json
import time
import random

# Sample DataFrame
# df = pd.read_parquet('path/to/your/parquet/file')


# List of client credentials
client_credentials = [
    {"client_id": id_1, "client_secret": sc_1},
    {"client_id": id_2, "client_secret": sc_2},
    {"client_id": id_3, "client_secret": sc_3},
]

def search_api(client_id, client_secret, query):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/shop?query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            return json.loads(response_body.decode('utf-8'))
        else:
            print("Error Code:", rescode)
            return None
    except urllib.error.HTTPError as e:
        if e.code == 429:
            print("HTTPError 429: API rate limit exceeded")
            return 429
        else:
            print("HTTPError:", e.code)
            return None
    except urllib.error.URLError as e:
        print("URLError:", e.reason)
        return None
    except Exception as e:
        print("General Error:", str(e))
        return None
    
def exponential_backoff(retries):
    delay = (2 ** retries) + random.uniform(0, 1)
    print(f"Exponential backoff: retry {retries}, sleeping for {delay:.2f} seconds")
    time.sleep(delay)

# Iterate over search terms and perform API search
current_credentials = 0
results = {}
search_terms = list(set(df[product_col].tolist()))

for term in tqdm(search_terms):
    retries = 0
    while retries < 5:  # Retry up to 5 times
        client_id = client_credentials[current_credentials]["client_id"]
        client_secret = client_credentials[current_credentials]["client_secret"]
        
        result = search_api(client_id, client_secret, term)
        
        if result == 429:
            current_credentials = (current_credentials + 1) % len(client_credentials)
            if current_credentials == 0:
                print("All credentials exhausted. Waiting for 60 seconds before retrying...")
                time.sleep(60)
            retries += 1
            exponential_backoff(retries)
        else:
            if result:
                items = result.get('items', [])
                if len(items) > 0:
                    ct1 = items[0]['category1']
                    ct2 = items[0]['category2']
                    ct3 = items[0]['category3']
                    ct4 = items[0]['category4']
                    results[term] = [ct1, ct2, ct3, ct4]
                else:
                    results[term] = [None, None, None, None]
            else:
                results[term] = [None, None, None, None]
            break
        
        time.sleep(1)  # Add a delay between retries

result_df = pd.DataFrame.from_dict(results, orient='index', columns=['category1', 'category2', 'category3', 'category4'])
result_df.reset_index(inplace=True)
result_df.rename(columns={'index': 'name'}, inplace=True)


HTTPError 429: API rate limit exceeded
Exponential backoff: retry 1, sleeping for 2.18 seconds


In [47]:
result_df

,name,category1,category2,category3,category4
0,Ottocast Play2Video Pro무선 안드로이드 오토 앤 애플 카플레이 올...,디지털/가전,자동차기기,내비게이션/액세서리,내비게이션액세서리
1,블루존 오뚜기 큰밥 300g x 5개 +오뚜기 흰밥 200g x 5개 세트,None,None,None,None
2,크라프트 각대 종이 봉투 중,생활/건강,문구/사무용품,카드/엽서/봉투,봉투
3,윈프라이스 쇼 케이스 가방 보관함,생활/건강,수납/정리용품,정리함,소품정리함
4,"머지 더 퍼스트 펜 아이라이너 0.5g, 오레오, 2개",화장품/미용,색조메이크업,아이라이너,펜슬형
...,...,...,...,...,...
35229,알뜨리 보온 보냉백,스포츠/레저,캠핑,아이스박스,
35230,원목수납침대 리브우드 LED 헤드 3단수납침대 슈퍼싱글침대 퀸침대,가구/인테리어,침실가구,침대,침대프레임
35231,모토벨로 TX8 프로3 DUAL 500W 48V 15Ah 전기자전거,스포츠/레저,자전거,자전거/MTB,전기자전거
35232,에이플라임 포인트샷 골프거리측정기,스포츠/레저,골프,골프필드용품,거리측정기


In [53]:
print(len(df))

merged_df = df.merge(result_df,how='left',on='name')
print(len(df))

66998
66998


In [59]:
merged_df.to_csv("../result/2024-06-01_2024-07-15_merged.csv",encoding='utf-8-sig')

In [61]:
merged_df.columns

Index(['Event_Date', 'Cookie_ID', 'Platform', 'Event_Category',
       'Client_IP_subdivision', 'Client_IP_City', 'Label', 'Action', 'Value',
       'productListID', 'query', 'IsRenewal', 'name', 'price', 'position',
       'category1', 'category2', 'category3', 'category4'],
      dtype='object')